In [ ]:
from spatiallyAdaptiveSingleDimension2 import SpatiallyAdaptiveSingleDimensions2
%matplotlib inline
import matplotlib.pyplot as plt
from sys import path
import numpy as np

#from numpy import *
import scipy as sp

from PerformTestCase import *
from Function import *
from ErrorCalculator import *
import math

# evaluation points for error estimation
def evaluation_points(a, b, d):
    grid = np.meshgrid(*(np.linspace(a[i], b[i], 10) for i in range(len(a))))
    points = list(zip(*(x.flat for x in grid)))

    return points


# shortcut for romberg SASD
def get_single_dim_romberg(a, b, f, reference_solution,
                           slice_grouping, slice_version, container_version,
                           force_full_binary_tree_grid):
    grid_romberg = GlobalRombergGrid(a=a, b=b, modified_basis=False, boundary=True,
                                     slice_grouping=slice_grouping,
                                     slice_version=slice_version,
                                     container_version=container_version)

    operation_romberg = Integration(f=f, grid=grid_romberg, dim=dim, reference_solution=reference_solution)

    return SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_romberg, rebalancing=False,
                                              force_full_binary_tree_grid=force_full_binary_tree_grid)

# Settings
dim = 2
a = np.zeros(dim)
b = np.ones(dim)
max_tol = 20  # set to max, for better comparison
max_evaluations = 10 ** 2
evaluation_points = evaluation_points(a, b, dim)

# --- Smooth functions

# coeffs = np.ones(dim)
# midpoint = np.ones(dim) * 0.5
# f = GenzProductPeak(coeffs, midpoint)

coeffs = np.ones(dim)
f = GenzCornerPeak(coeffs=coeffs)

# coeffs = np.ones(dim)
# offset = 1
# f = GenzOszillatory(coeffs, offset)

# coeffs = np.ones(dim)
# midpoint = np.ones(dim) * 0.5
# f = GenzGaussian(coeffs, midpoint)

# --- Genz discontinuous

# midpoint = np.ones(dim) * 0.5
# coefficients = np.array([10**0 * (d+1) for d in range(dim)])

# f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# f = GenzDiscontinious2(border=midpoint,coeffs=coefficients)

# Plot
f.plot(np.ones(dim)*a,np.ones(dim)*b)
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# Trapezoidal Grid
grid_trapezoidal = GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)
operation_trapezoidal = Integration(f=f, grid=grid_trapezoidal, dim=dim, reference_solution=reference_solution)
trapezoidal = SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_trapezoidal, rebalancing=False,
                                                 force_full_binary_tree_grid=False)
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# grids for standard combi
grid = TrapezoidalGrid(a, b, boundary=True)
grid_gauss = GaussLegendreGrid(a=a, b=b)

Unit Slices

In [ ]:
r_unit = get_single_dim_romberg(a, b, f, reference_solution,
                                slice_grouping=SliceGrouping.UNIT,
                                slice_version=SliceVersion.ROMBERG_DEFAULT,
                                container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                force_full_binary_tree_grid=False)

r_full_unit = get_single_dim_romberg(a, b, f, reference_solution,
                                     slice_grouping=SliceGrouping.UNIT,
                                     slice_version=SliceVersion.ROMBERG_DEFAULT,
                                     container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                     force_full_binary_tree_grid=True)

r_trapezoidal = get_single_dim_romberg(a, b, f, reference_solution,
                                       slice_grouping=SliceGrouping.UNIT,
                                       slice_version=SliceVersion.TRAPEZOID,
                                       container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                       force_full_binary_tree_grid=False)

Slice Grouping

In [ ]:
r_grouped = get_single_dim_romberg(a, b, f, reference_solution,
                                   slice_grouping=SliceGrouping.GROUPED,
                                   slice_version=SliceVersion.ROMBERG_DEFAULT,
                                   container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                   force_full_binary_tree_grid=False)

r_full_grouped = get_single_dim_romberg(a, b, f, reference_solution,
                                        slice_grouping=SliceGrouping.GROUPED,
                                        slice_version=SliceVersion.ROMBERG_DEFAULT,
                                        container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                        force_full_binary_tree_grid=True)

Optimized slice grouping

In [ ]:
r_grouped_optimized = get_single_dim_romberg(a, b, f, reference_solution,
                                             slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                             slice_version=SliceVersion.ROMBERG_DEFAULT,
                                             container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                             force_full_binary_tree_grid=False)

r_full_grouped_optimized = get_single_dim_romberg(a, b, f, reference_solution,
                                                  slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                                  slice_version=SliceVersion.ROMBERG_DEFAULT,
                                                  container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                                  force_full_binary_tree_grid=True)

Unit Slices with sliced Romberg

In [ ]:

r_unit_sliced = get_single_dim_romberg(a, b, f, reference_solution,
                                       slice_grouping=SliceGrouping.UNIT,
                                       slice_version=SliceVersion.ROMBERG_SLICED,
                                       container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                       force_full_binary_tree_grid=False)

r_full_unit_sliced = get_single_dim_romberg(a, b, f, reference_solution,
                                            slice_grouping=SliceGrouping.UNIT,
                                            slice_version=SliceVersion.ROMBERG_SLICED,
                                            container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                            force_full_binary_tree_grid=True)

Compare all variants

In [ ]:
algorithmArray = [
    (trapezoidal, 1, 2, errorOperator, 'Trapezoidal Grid'),

    # (r_trapezoidal, 1, 2, errorOperator, 'Romberg Grid (Unit, Trapezoid)'),

    # (r_unit, 1, 2, errorOperator, 'Romberg Grid (Unit, Default)'),
    (r_full_unit, 1, 2, errorOperator, 'Romberg Grid (Full, Unit, Default)'),

    # (r_unit_sliced, 1, 2, errorOperator, 'Romberg Grid (Unit, Sliced)'),
    # (r_full_unit_sliced, 1, 2, errorOperator, 'Romberg Grid (Full, Unit, Sliced)'),

    # (r_grouped, 1, 2, errorOperator, 'Romberg Grid (Grouped, Default)'),
    (r_full_grouped, 1, 2, errorOperator, 'Romberg Grid (Full, Grouped, Default)'),

    # TODO check code of optimized
    # (r_grouped_optimized, 1, 2, errorOperator, 'RombergGrid (Grouped Optimized, Default)'),
    # (r_full_grouped_optimized, 1, 2, errorOperator, 'RombergGrid (Full, Grouped Optimized, Default)'),
]

performTestcaseArbitraryDim(f, a, b, algorithmArray,
                            max_tol, dim, 6, grid=grid_gauss, evaluation_points=evaluation_points,
                            max_evaluations=max_evaluations,
                            save_with_name="full_comparison",
                            calc_standard_schemes=True, # enable for standard schemes
                            minLmin=1,
                            maxLmin=3,
                            standard_combi_grid_name="Gauss-Legendre Grid"
                            )